In [1]:
import nltk
import pandas as pd
from nltk.tokenize import word_tokenize
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation, NMF
import string
import re
from datetime import datetime

In [2]:
df = pd.read_csv('processed.csv',index_col=False)
df.head(3)

,Unnamed: 0,id,num_comments,score,subreddit,upvote_ratio,all_text,Datetime,hour,Class,body_len,text_no_emojis,language,cleaned_tokenized_text,lemmatized_text
0,0,x2wyl4,1.0,1.0,domesticviolence,1.0,I can't even eat chocolate without thinking ab...,2022-09-01 10:22:54,10,1,356,I can't even eat chocolate without thinking ab...,en,"['eat', 'chocolate', 'thinking', 'orderedeaten...","['eat', 'chocolate', 'think', 'orderedeaten', ..."
1,1,x2uv4c,1.0,1.0,domesticviolence,1.0,How do I know he's really changing? I think th...,2022-09-01 08:42:03,8,1,380,How do I know he's really changing? I think th...,en,"['changing', 'think', 'time', 'changing', 'tim...","['change', 'think', 'time', 'change', 'time', ..."
2,2,x2pn4w,1.0,1.0,domesticviolence,1.0,Just got my 5th NFA 😭 This is more of a rant t...,2022-09-01 04:46:37,4,1,732,Just got my 5th NFA :loudly_crying_face: This ...,en,"['nfa', 'loudly_crying_face', 'rant', 'broken'...","['nfa', 'loudly_crying_face', 'rant', 'broken'..."


Compare and test LDA and NMF performance on domesticviolence subset

In [ ]:
dv= df.loc[df['subreddit'] == 'domesticviolence']

In [ ]:
#starting to experiment with CountVectorizer(max_df=0.8, min_df=5, ngram_range=(1,2))
count_vect_0 = CountVectorizer(max_df=0.8, min_df=5, ngram_range=(1,2), stop_words='english')
dv_doc_term_matrix_0 = count_vect_0.fit_transform(dv['lemmatized_text'].values.astype('U'))

In [ ]:
LDA_3_topics= LatentDirichletAllocation(n_components=3,max_iter=50)
dv_LDA_0 = LDA_3_topics.fit(dv_doc_term_matrix_0)

In [ ]:
#Print the 10 words with highest probabilities for all the topics
for i,topic in enumerate(dv_LDA_0.components_):
    print(f'Top 10 words for topic #{i}:')
    print([count_vect_0.get_feature_names_out()[i] for i in topic.argsort()[-10:]])

Top 10 words for topic #0:
['aware past', 'adult daughter', 'agree live', 'american way', 'attractive', 'baby wongs', 'block unblocked', 'bother write', 'abhors', 'anytime thing']
Top 10 words for topic #1:
['agh', 'attempt head', 'arm lay', 'bf country', 'advice friendship', 'behave inappropriately', 'bit fall', 'ask ear', 'block unblocked', 'big struggle']
Top 10 words for topic #2:
['approve try', 'abhors', 'asks rabbit', 'bother write', 'bar misc', 'bf country', 'big struggle', 'block unblocked', 'ask ear', 'amp sort']


In [ ]:
#Print the 20 words with highest probabilities for all the topics for dv
for i,topic in enumerate(dv_LDA_0.components_):
    print(f'Top 20 words for topic #{i}:')
    print([count_vect_0.get_feature_names_out()[i] for i in topic.argsort()[-20:]])

Top 20 words for topic #0:
['agree man', 'approve try', 'alloy', 'able relate', 'bit fall', 'boring ask', 'arm lay', 'anime character', 'big struggle', 'bless rain', 'aware past', 'adult daughter', 'agree live', 'american way', 'attractive', 'baby wongs', 'block unblocked', 'bother write', 'abhors', 'anytime thing']
Top 20 words for topic #1:
['australia btw', 'best employee', 'account cause', 'age decide', 'boring ask', 'apartment people', 'apologize stop', 'bother write', 'bastard child', 'apart thats', 'agh', 'attempt head', 'arm lay', 'bf country', 'advice friendship', 'behave inappropriately', 'bit fall', 'ask ear', 'block unblocked', 'big struggle']
Top 20 words for topic #2:
['agh', 'best employee', 'away reason', 'body healthy', 'able relate', 'anime character', 'ampxb believe', 'ask big', 'bit fall', 'bf shower', 'approve try', 'abhors', 'asks rabbit', 'bother write', 'bar misc', 'bf country', 'big struggle', 'block unblocked', 'ask ear', 'amp sort']


In [ ]:
# I don't see any keyowrds indicating violence and abuse, and those two grams don't make much sense, try unigram vectorizer
#experiemnt with CountVectorizer(max_df=0.95, min_df=2, ngram_range=(1,1))
count_vect_1 = CountVectorizer(max_df=0.9, min_df=3, ngram_range=(1,1), stop_words='english')
dv_doc_term_matrix_1 = count_vect_1.fit_transform(dv['lemmatized_text'].values.astype('U'))
non_dv_doc_term_matrix_1 = count_vect_1.fit_transform(non_dv['lemmatized_text'].values.astype('U'))

In [ ]:
LDA_3_topics = LatentDirichletAllocation(n_components=3, max_iter=50)
dv_LDA_1 = LDA_3_topics.fit(dv_doc_term_matrix_1)

In [ ]:
#Print the 10 words with highest probabilities for all the topics for dv
for i,topic in enumerate(dv_LDA_1.components_):
    print(f'Top 10 words for topic #{i}:')
    print([count_vect_1.get_feature_names_out()[i] for i in topic.argsort()[-10:]])

Top 10 words for topic #0:
['cavy', 'calcium', 'aaand', 'cozied', 'eap', 'directionless', 'disciplinary', 'drift', 'cash', 'bereft']
Top 10 words for topic #1:
['backpacking', 'cocktail', 'cisco', 'anon', 'bellow', 'drift', 'confection', 'eap', 'aaand', 'boyfriendgirlfriend']
Top 10 words for topic #2:
['assistant', 'brand', 'carlsberg', 'directionless', 'aot', 'disrupt', 'demonstrative', 'drift', 'cash', 'disciplinary']


In [ ]:
LDA_5_topics = LatentDirichletAllocation(n_components=5, max_iter=50)
dv_LDA_2 = LDA_5_topics.fit(dv_doc_term_matrix_1)

In [ ]:
#Print the 20 words with highest probabilities for all the five topics 
for i,topic in enumerate(dv_LDA_2.components_):
    print(f'Top 20 words for topic #{i}:')
    print([count_vect_1.get_feature_names_out()[i] for i in topic.argsort()[-20:]])

Top 20 words for topic #0:
['berate', 'conducive', 'artillery', 'ebbed', 'deja', 'elate', 'clunky', 'anniversary', 'battery', 'cryptid', 'dylan', 'abandonment', 'antigay', 'assthats', 'abandon', 'commenter', 'eachother', 'consume', 'barberqueue', 'britain']
Top 20 words for topic #1:
['amplifier', 'depressant', 'clunky', 'bruh', 'augustseptember', 'abandon', 'buff', 'celiac', 'biomedical', 'degrades', 'ebbed', 'comprehend', 'catalog', 'anxietyinducing', 'clergyman', 'auction', 'elm', 'britain', 'bindr', 'clawed']
Top 20 words for topic #2:
['distinctive', 'accompany', 'abba', 'actualy', 'dopey', 'celiac', 'abandon', 'cross', 'checker', 'cellar', 'aum', 'cervical', 'cleanup', 'caretaking', 'cgpa', 'berate', 'djok', 'ebbed', 'elm', 'bismol']
Top 20 words for topic #3:
['confucius', 'angular', 'cypriot', 'buff', 'bruh', 'biggie', 'bard', 'advicesupport', 'distinctive', 'darken', 'celiac', 'aum', 'cockney', 'brotherm', 'ebbed', 'cgpa', 'dopey', 'apron', 'devestated', 'djok']
Top 20 words f

In [ ]:
LDA_10_topics = LatentDirichletAllocation(n_components=10, max_iter=50)
dv_LDA_3 = LDA_10_topics.fit(dv_doc_term_matrix_1)

In [ ]:
#Print the 20 words with highest probabilities for all the five topics 
for i,topic in enumerate(dv_LDA_3.components_):
    print(f'Top 20 words for topic #{i}:')
    print([count_vect_1.get_feature_names_out()[i] for i in topic.argsort()[-20:]])

Top 20 words for topic #0:
['cleanup', 'adversity', 'correspond', 'abandon', 'adviser', 'ccepted', 'djok', 'black_mediumsmall_squareim', 'anniversary', 'ebbed', 'abandonment', 'cupid', 'elm', 'cryptid', 'artillery', 'antigay', 'consume', 'berate', 'assthats', 'commenter']
Top 20 words for topic #1:
['breaker', 'cleanup', 'berate', 'devestated', 'ecret', 'dishonesty', 'abba', 'checker', 'abandon', 'dopey', 'bobcat', 'distinctly', 'caretaking', 'elm', 'cross', 'distinctive', 'djok', 'ebbed', 'cgpa', 'bismol']
Top 20 words for topic #2:
['bothersome', 'albino', 'apron', 'chaste', 'centre', 'bshells', 'dickens', 'abandon', 'aum', 'ecret', 'boeing', 'distracts', 'cgpa', 'elate', 'emanate', 'acute', 'conducive', 'checker', 'cervical', 'actualy']
Top 20 words for topic #3:
['barbarian', 'djok', 'buff', 'cgpa', 'depressant', 'abandon', 'britain', 'augustseptember', 'ebbed', 'amplifier', 'anxietyinducing', 'catalog', 'comprehend', 'biomedical', 'elm', 'bindr', 'degrades', 'clergyman', 'auction'

all the models so far are not satisfactory, because they don't converge on topics that indicate violence. let's try NMF.

In [ ]:
tfidf_vectorizer = TfidfVectorizer(max_df=0.95, min_df=2)
dv_tfidf_0 = tfidf_vectorizer.fit_transform(df['lemmatized_text'].values.astype('U'))

In [ ]:
dv_nmf_0 = NMF(n_components=3, random_state=1, alpha=.1, l1_ratio=.5, init='nndsvd').fit(dv_tfidf_0)

/usr/local/lib/python3.8/dist-packages/sklearn/decomposition/_nmf.py:1422: FutureWarning: `alpha` was deprecated in version 1.0 and will be removed in 1.2. Use `alpha_W` and `alpha_H` instead
  warnings.warn(


In [ ]:
#Print the 20 words with highest probabilities for all the topics 
for i,topic in enumerate(dv_nmf_0.components_):
    print(f'Top 20 words for topic #{i}:')
    print([tfidf_vectorizer.get_feature_names_out()[i] for i in topic.argsort()[-20:]])

Top 20 words for topic #0:
['talk', 'come', 'need', 'love', 'day', 'think', 'relationship', 'help', 'start', 'ive', 'abuse', 'friend', 'leave', 'try', 'year', 'thing', 'time', 'feel', 'want', 'like']
Top 20 words for topic #1:
['school', 'old', 'sibling', 'kid', 'abuse', 'doesnt', 'hit', 'child', 'home', 'live', 'house', 'help', 'family', 'parent', 'father', 'mother', 'brother', 'sister', 'dad', 'mom']
Top 20 words for topic #2:
['long', 'jail', 'relationship', 'away', 'need', 'story', 'dv', 'change', 'ex', 'post', 'boyfriend', 'abusive', 'victim', 'advice', 'abuse', 'violence', 'abuser', 'domestic', 'help', 'delete']


NMF is so much faster and the results are so much better! the 3rd topic is clearly about abuse and domestic violence. 

### Conclusion:
1. Gensim gave better results, it captured domestic violence topics every time while sklearn failed most of time.
2. Both unigram and bigram were experimented in sklearn, didn’t work well. NMF(Non-Negative Matrix) in sklearn, on other hand gave a better result, and it was much faster.